## Discovery and Ordering Workflow

This sample workflow will guide a user through using the Discovery and Ordering class of the Maxar Geospatial Platform SDK with real world examples. The process will go through:
* Searching STAC items
* Viewing pipelines
* Validating an order
* Checking an order's estimated usage
* Placing an order
* Canceling an order

#### Initialize the Maxar Geospatial Platform SDK Module

In [ ]:
from MGP_SDK.interface import Interface

try:
  interface = Interface() # if .MGP-config was created
except:
  interface = Interface('<user_name>','<user_password>', '<user_client_id>') # if .MGP-config was not created

#### Initialize the discovery and ordering portions of the Maxar Geospatial Platform SDK

In [ ]:
mgp_discovery = interface.discovery_service
mgp_ordering = interface.order_service

#### Discover STAC items

The first thing to do is search the available STAC items via Discovery. Here some variables will be set up to make the discovery process more manageable.

In [ ]:
bbox = "-105,40,-104,41"
datetime = "2015-01-01T00:00:00Z/2016-01-01T00:00:00Z"
collections = "wv02"
where = "eo:cloud_cover<20"
limit = 10

The `stac_search` function will be utilized and a list of all of the ids that match the desired criteria will be iterated through and a variable for the first id in that list will be set.

In [ ]:
features = mgp_discovery.stac_search(bbox=bbox, datetime=datetime, where=where, collections=collections, orderby='id', limit=limit)
features_list = []
for feature in features['features'][:limit]:
    features_list.append(feature['id'])
wv02_feature = features_list[0]
print(wv02_feature)

Now that a single feature has been selected, the metadata can be narrowed down for just that feature with the `get_stac_item` funciton. The supported pipeline for this feature needs to be known, so the result will be narrowed down to show the pipelines and their names.

In [ ]:
desired_feature = mgp_discovery.get_stac_item(collection_id=collections, item_id=wv02_feature)
for i in desired_feature['links']:
    print(i['rel'])
    print(i['href'])

For this workflow, the `map-ready` pipeline will be utilized. The above cell shows that the feature can be ordered via the `map-ready` pipeline.

If a user is unfamiliar with a pipeline and what is needed for the order to work, the `get_pipeline_details` function can be used to determine what information needs to be passed in. The namespace of the pipeline can be found in the `href` link from the above function. It is the section of the link between `pipelines` and the pipeline name, in this case, with a namespace of `imagery` and a name of `map-ready`. With the pipeline identified, the response of the `get_pipeline_details` function can be narrowed down to show the necessary information needed to place an order.

In [ ]:
pipeline = mgp_ordering.get_pipeline_details(namespace='imagery', name='map-ready')
required_info = pipeline['data']['settings_schema']['required']
print("The required information needed to place an order on this pipeline is: {}".format(required_info))
for info in required_info:
    type_info = pipeline['data']['settings_schema']['properties'][info]
    print("The type of data for {} is: {}".format(info, type_info))

From this response it can determined that the `inventory_ids` and the `customer_description` are required for order placement, and that `inventory_ids` is an array of strings while `customer_description` is a string.

#### Ordering a feature

Before an order can be fully placed, the order will need to be verified so that the order is formatted properly and will be accepted, as well as checking what the estimated cost of the order will be. First some variables will need to be set to make the requests more manageable. The `output_config` variable will be a dictionary where the S3 information is stored, the notificaitons variable will be a list of a dictionary that will store email information, and the metadata variable will be a dictionary that stores the `project_id` (order name).

In [ ]:
namespace = 'imagery'
name = 'map-ready'
output_config = {"output_config": {"amazon_s3": {"bucket": "yourS3BucketName", "prefix": "your/S3/bucket/prefix/name"}}}
settings = {
            "settings": {
                "inventory_ids": [wv02_feature],
                "customer_description": "your order description"
            }
        }
notifications = [{"type": "email", "target": "your.email@address.com", "level": "FINAL_ONLY"}]
metadata = {"metadata": {"project_id": "your order name"}}

The order can now be validated before placing it by setting the validate argument to `True`. *Note: The response can vary by the type of content ordered*

In [ ]:
order_validate = mgp_ordering.place_order(namespace=namespace, name=name, output_config=output_config, settings=settings, 
                                         notifications=notifications, metadata=metadata, validate=True)
print(order_validate)

The status of the order is shown as `RECEIVED`, indicating that the order passes the validation, meaning the order can be placed. One last thing to check before the order is placed is to check the estimated usage of the order. *Note: The response can vary by the type of content ordered*

In [ ]:
order_estimate = mgp_ordering.get_usage_estimate(namespace=namespace, name=name, output_config=output_config, settings=settings, 
                                         notifications=notifications, metadata=metadata)
print(order_estimate)

The response of the above cell shows that the order will use `1922.748 sqkm` of usage. The order can now be placed.

In [ ]:
order = mgp_ordering.place_order(namespace=namespace, name=name, output_config=output_config, settings=settings, 
                                         notifications=notifications, metadata=metadata)
order_id = order['data']['id']
print(order)

After the order has been placed, a user may want to check the full details of the submitted order. This can be done with the `get_order_details` function and passing in the order id generated from the placement of the order.

In [ ]:
check_order = mgp_ordering.get_order_details(order_id=order_id)
print(check_order)

A user also may want to occasionally check up on the status of an order. This can be done by utilizing the `get_order_events` function and checking the message of the response.

In [ ]:
check_order_events = mgp_ordering.get_order_events(order_id=order_id)
print(check_order_events)

In some instances, a user may want to cancel an order. The `cancel_order` function can be utilized to cancel the desired order. Orders can be cancelled until the delivery process begins.

In [ ]:
cancel_order = mgp_ordering.cancel_order(order_id)
print(cancel_order)